In [1]:
import csv
import numpy as np
import os
import pandas as pd
import requests
import time

In [2]:
#<Path>
#path = '/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/5_Similarity/'
#file_name = ['sorted_score_vector_except_1_aa', 'sorted_score_vector_except_1_ab', 'sorted_score_vector_except_1_ac', 'sorted_score_vector_except_1_ad', 'sorted_score_vector_except_1_ae', 'sorted_score_vector_except_1_af', 'sorted_score_vector_except_1_ag', 'sorted_score_vector_except_1_ah', 'sorted_score_vector_except_1_ai', 'sorted_score_vector_except_1_aj']
#score_path = list()
#for i in file_name:
#    score_path.append(path + i)

In [3]:
#def file_open(file_path):
#    print time.strftime('%a %H:%M:%S')
#    data = []
#    with open(file_path) as f:
#        for line in f:
#            data.append(line.split(','))
#    print time.strftime('%a %H:%M:%S')
#    return data

In [4]:
#score = list()
#for i in score_path:
#    score = score + file_open(i)
#    print 'check'

Sun 21:14:59
Sun 21:16:14
check
Sun 21:16:16
Sun 21:17:30
check
Sun 21:17:33
Sun 21:18:54
check
Sun 21:18:59
Sun 21:20:55
check
Sun 21:21:04
Sun 21:23:36
check
Sun 21:23:50
Sun 21:26:31
check
Sun 21:26:45
Sun 21:29:13
check
Sun 21:29:30
Sun 21:30:25
check
Sun 21:30:51
Sun 21:34:06
check
Sun 21:34:25
Sun 21:35:40
check


In [2]:
#확인
#f = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/5_Similarity/5_sorted_score_vector_except_1(descending)_no_score.csv', 'r')
#f = open('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/5_sorted_score_vector_except_1(descending)_no_score.csv', 'r')
f = open('/home/share/aschoi/data/COCONUT_Herbal/5_Similarity/7_sorted_score_vector_except_1(descending)_top_10_percent_no_score.csv', 'r')
score = []
print time.strftime('%a %H:%M:%S')
while True:
    line = f.readline()
    if not line: break
    score.append(line.split(','))
print time.strftime('%a %H:%M:%S')
f.close()

Mon 21:21:36
Mon 21:22:47


In [3]:
len(score) #multiple file은 646688125임.

64664705

In [4]:
# aceept user input
cutoff_percentage = input('Enter the percentage number (ex. 0.1 %) : ')

Enter the percentage(ex. 0.1) : 0.1


In [5]:
# extract element over cutoff. cutoff is top 0.0001 per cent of wo1.
WHOLE_LENGTH = 646647053
cutoff_idx = int(WHOLE_LENGTH*cutoff_percentage / 100)
#cutoff = score[cutoff_idx] # 0.9939

similar_comp = []
#for item2 in score[:cutoff_idx]: 
#    similar_comp.append(item2)
similar_comp = score[:cutoff_idx]

In [6]:
print len(similar_comp)
#print cutoff

646647


In [7]:
def read_delimited_file(f,delimiter):
    f.seek(0)
    temp = []
    print time.strftime('%a %H:%M:%S')
    while True:
        line = f.readline()
        if not line: break
        temp.append(line.split(delimiter))
    f.close()
    print time.strftime('%a %H:%M:%S')
    return temp

In [8]:
#df_hmdb = pd.read_table('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/HMDB_PubChem_KEGG.tsv')
df_hmdb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/4_HMDB/HMDB_PubChem_KEGG.tsv')

In [9]:
df_hmdb.head()

,Accession,Name,PubChem_CID,KEGG_ID
0,HMDB00002,"1,3-Diaminopropane",428.0,C00986
1,HMDB00005,2-Ketobutyric acid,58.0,C00109
2,HMDB00010,2-Methoxyestrone,440624.0,C05299
3,HMDB00011,(R)-3-Hydroxybutyric acid,92135.0,C01089
4,HMDB00012,Deoxyuridine,13712.0,C00526


In [10]:
#df_kegg_rpair = pd.read_table('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')
df_kegg_rpair = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_Rpair.txt')

In [11]:
df_kegg_rpair.head()

,Compound1,Compound2
0,C00005,C00006
1,C00003,C00004
2,C00002,C00008
3,C00019,C00021
4,C00007,C00027


In [12]:
similar_comp[:10]

[['80303', 'HMDB37761\r\n'],
 ['69881', 'HMDB02174\r\n'],
 ['69881', 'HMDB02308\r\n'],
 ['69881', 'HMDB03458\r\n'],
 ['69881', 'HMDB06316\r\n'],
 ['69881', 'HMDB14345\r\n'],
 ['70406', 'HMDB30251\r\n'],
 ['45577', 'HMDB30734\r\n'],
 ['45577', 'HMDB34095\r\n'],
 ['63987', 'HMDB15185\r\n']]

In [13]:
#top herb id, top hmdb id
top_herb = list()
top_hmdb = list()
for i in similar_comp:
    top_herb.append(int(i[0]))
    top_hmdb.append(i[1].rstrip('\r\n'))

In [14]:
top_hmdb[0:5]

['HMDB37761', 'HMDB02174', 'HMDB02308', 'HMDB03458', 'HMDB06316']

In [15]:
# top hmdb의 kegg id를 찾는다.
flag_top_hmdb = df_hmdb['Accession'].isin(top_hmdb)
df_top_hmdb_keggid = df_hmdb[flag_top_hmdb] #이 데이터 프레임은 kegg id를 갖고 있는 top hmdb이다.

In [16]:
df_top_hmdb_keggid.head()

,Accession,Name,PubChem_CID,KEGG_ID
0,HMDB00002,"1,3-Diaminopropane",428.0,C00986
1,HMDB00005,2-Ketobutyric acid,58.0,C00109
2,HMDB00010,2-Methoxyestrone,440624.0,C05299
3,HMDB00011,(R)-3-Hydroxybutyric acid,92135.0,C01089
6,HMDB00016,Deoxycorticosterone,6166.0,C03205


In [17]:
# kegg id를 갖고 있는 top hmdb 개수
len(df_top_hmdb_keggid)

1130

In [18]:
#top hmdb가 rpair의 substrate로 있는 rpair들의 찾는다.
flag_top_hmdb_substrate = df_kegg_rpair['Compound1'].isin(df_top_hmdb_keggid['KEGG_ID']) 
df_top_hmdb_rpair = df_kegg_rpair[flag_top_hmdb_substrate] # 이 데이터 프레임은 top hmdb가 rpair의 substrate로 있는 rpair들이다.

In [19]:
# top hmdb 중에 rpair의 substrate로 있는 애들 갯수
len(df_top_hmdb_rpair['Compound1'].drop_duplicates())

542

In [20]:
# top hmdb가 rpair의 substrate로 있는 pair의 개수
len(df_top_hmdb_rpair)

3317

In [21]:
df_top_hmdb_rpair.head()

,Compound1,Compound2
1,C00003,C00004
2,C00002,C00008
6,C00002,C00020
7,C00002,C00013
8,C00002,C00009


In [22]:
#df_herb = pd.read_table('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/2-4_herb_cas_pubchem_2_except_null.tsv', index_col = 0)
df_herb = pd.read_table('/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/2-4_herb_cas_pubchem_2_except_null.tsv', index_col = 0)

In [23]:
df_herb.head()

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
46,"[1, 6, 8, 78, 155, 174, 247, 344, 357, 369, 45...","['Melia azedarach', 'Atractylodes lancea', 'Sc...",32469,"1,3,11-Elematriene",10583,33880-83-0,null,null
62,"[1, 3, 8, 127, 175, 202, 364, 383, 489, 526, 6...","['Melia azedarach', 'Paeonia lactiflora', 'Scu...",10237,Phenol,20488062,null,CHEMBL14060,null
65,"[1, 3, 127, 175, 383, 493, 526, 639, 1226, 129...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",1707,3-Methylphenol,342,108-39-4,null,null
66,"[1, 3, 8, 127, 175, 254, 383, 493, 526, 639, 1...","['Melia azedarach', 'Paeonia lactiflora', 'Scu...",1622,4-Methylphenol,2879,106-44-5,null,null
69,"[1, 3, 127, 175, 247, 254, 326, 357, 383, 489,...","['Melia azedarach', 'Paeonia lactiflora', 'Cam...",5525,Benzyl alcohol,244,100-51-6,null,null


In [24]:
#top herb의 pubchem id를 찾는다.
flag_top_herb = df_herb['compID'].isin(top_herb)
df_top_herb_pubchemid = df_herb[flag_top_herb] #이 데이터 프레임은 pubchem id를 갖고 있는 top herb이다.

In [25]:
df_top_herb_pubchemid.head()

,herbID,herbName,compID,compName,pubchemID,casID,chemblID,stitchID
79,"[1, 168, 2328]","['Melia azedarach', 'Melia azed-arach', 'Dicta...",37772,"(3R,3aR)-3-(3-Furyl)-3a,7-dimethyl-3a,4,5,6-te...",124039,28808-62-0,null,null
93,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",33032,"(3beta,5xi,9xi,13xi,14beta,17alpha,20S,23R,24R...",177786,16838-01-0,null,null
95,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",33034,"10-(3-Furyl)-4,4,6a,10a,12b-pentamethyl-3,8-di...",12004512,2753-30-2,null,null
118,"[1, 168]","['Melia azedarach', 'Melia azed-arach']",33179,"16-Hydroxylanosta-7,24-dien-3-one",88810,21688-61-9,null,null
122,"[1, 168, 245, 502, 794, 1477]","['Melia azedarach', 'Melia azed-arach', 'Lyciu...",32814,"9,19-Cyclolanostan-3-ol",313075,4657-58-3,null,null


In [109]:
len(df_top_herb_pubchemid)

6826

In [26]:
df_similar_comp = pd.DataFrame({'Accession':top_hmdb, 'compID' :top_herb})

In [27]:
df_similar_comp.head()

,Accession,compID
0,HMDB37761,80303
1,HMDB02174,69881
2,HMDB02308,69881
3,HMDB03458,69881
4,HMDB06316,69881


In [28]:
1. df_similar_comp에서 rpair의 substrate로 있는 hmdb들을 찾고 그에 맞는 compID를 찾는다.

SyntaxError: invalid syntax (<ipython-input-28-eaaa7eee1887>, line 1)

In [29]:
#kegg id를 가지는 top hmdb 중에 rpair의 sub로 있는 애들 찾기.
flag_hmdb_substrate = df_top_hmdb_keggid['KEGG_ID'].isin(df_top_hmdb_rpair['Compound1'])
df_hmdb_substrate = df_top_hmdb_keggid[flag_hmdb_substrate]

In [30]:
df_hmdb_substrate.head() # 543개임

,Accession,Name,PubChem_CID,KEGG_ID
0,HMDB00002,"1,3-Diaminopropane",428.0,C00986
1,HMDB00005,2-Ketobutyric acid,58.0,C00109
2,HMDB00010,2-Methoxyestrone,440624.0,C05299
3,HMDB00011,(R)-3-Hydroxybutyric acid,92135.0,C01089
6,HMDB00016,Deoxycorticosterone,6166.0,C03205


In [31]:
flag_similar_hmdb = df_similar_comp['Accession'].isin(df_hmdb_substrate['Accession'])
df_hmdb_substrate_similar = df_similar_comp[flag_similar_hmdb]

In [32]:
df_hmdb_substrate_similar.head() # 14129개임

,Accession,compID
9,HMDB15185,63987
61,HMDB14812,57126
133,HMDB03249,34964
141,HMDB03249,61907
149,HMDB03249,77622


In [33]:
len(df_hmdb_substrate_similar)

14127

In [34]:
df_hmdb_herb = pd.merge(df_hmdb_substrate, df_hmdb_substrate_similar, how = 'inner', on = ['Accession', 'Accession'])

In [35]:
df_hmdb_herb.head() # 14127개

,Accession,Name,PubChem_CID,KEGG_ID,compID
0,HMDB00002,"1,3-Diaminopropane",428.0,C00986,65801
1,HMDB00005,2-Ketobutyric acid,58.0,C00109,70898
2,HMDB00005,2-Ketobutyric acid,58.0,C00109,82614
3,HMDB00010,2-Methoxyestrone,440624.0,C05299,50190
4,HMDB00010,2-Methoxyestrone,440624.0,C05299,56503


In [36]:
print time.strftime('%a %H:%M:%S')
s_hmdb_herb_substrate = df_hmdb_herb['KEGG_ID'].drop_duplicates()
herb_product_pair = pd.DataFrame(columns = ['compID', 'product'] )
for i in s_hmdb_herb_substrate:
    df_hmdb_herb_kegg = df_hmdb_herb[df_hmdb_herb.KEGG_ID == i]
    df_hmdb_rpair_kegg = df_top_hmdb_rpair[df_top_hmdb_rpair.Compound1 == i]
    s_compID = df_hmdb_herb_kegg['compID']
    s_product = df_hmdb_rpair_kegg['Compound2']
    herb_compID, kegg_product = pd.tools.util.cartesian_product([s_compID, s_product])
    temp_df = pd.DataFrame(dict(compID= herb_compID, product = kegg_product))
    herb_product_pair = herb_product_pair.append(temp_df)
print time.strftime('%a %H:%M:%S')

Mon 21:24:08
Mon 21:24:09


In [117]:
type(herb_compID)

numpy.ndarray

In [112]:
s_compID

14063    31379
14064    31299
14065    44274
14066    53310
14067    76222
14068    32055
14069    62598
14070    66423
14071    31653
14072    61818
14073    78938
14074    77058
14075    53315
14076    70791
14077    31960
14078    51497
14079     9332
14080    61087
14081    61086
14082    65006
14083    31538
14084    31166
14085    65054
14086    53467
14087    52108
14088    51434
14089    71463
14090    70113
14091    49586
14092    55086
         ...  
14097    31601
14098    38808
14099    53830
14100    53201
14101    67221
14102    31241
14103    71280
14104    72000
14105    55426
14106    31378
14107    31069
14108    31070
14109    31396
14110    54025
14111    40398
14112    31339
14113    31215
14114    31463
14115    31464
14116    60059
14117    40887
14118    70217
14119    70849
14120    63050
14121    65007
14122    60179
14123    61293
14124    63550
14125    68549
14126    63331
Name: compID, dtype: int64

In [37]:
herb_product_pair.head() #48865

,compID,product
0,65801.0,C05665
1,65801.0,C03375
2,65801.0,C03283
3,65801.0,C15853
4,65801.0,C01029


In [38]:
len(herb_product_pair)

48865

In [43]:
url

'http://rest.genome.jp/ezyme/CID82614/C02876'

In [100]:
cnt

46962

In [101]:
sub

79371

In [102]:
pro

'C16314'

In [103]:
from urllib2 import urlopen
import urllib2
import csv

#중간에 끊겨서 다시 한다!
#cnt = 17047
print time.strftime('%a %H:%M:%S')

with open("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_4_predicted_ontology_.tsv", "w") as resultFile:
    features = ['substrate', 'product', 'predicted_ontology', 'predicted_score']
    resultTsv = csv.DictWriter(resultFile, features, dialect=csv.excel_tab)
    resultTsv.writeheader()
    
    url_base = 'http://rest.genome.jp/ezyme/'
    for index, row in herb_product_pair[cnt:].iterrows():
        sub = int(row['compID'])
        pro = row['product']
        url = url_base + 'CID'+str(sub) +'/' + pro
        
        try:
            f = urlopen(url)
            all_data = f.readlines()
            data = all_data[1].split('\t')
            predicted_ontology = data[1]
            predicted_score = data[0]
        
            resultTsv.writerow({'substrate':sub, 'product':pro, 'predicted_ontology' : predicted_ontology, 'predicted_score' : predicted_score})
            
        except urllib2.HTTPError, e:
            cnt = cnt + 1
            continue
            
            
        cnt = cnt + 1
        if cnt % 1000 == 0 :
            print '%d : '% cnt + time.strftime('%a %H:%M:%S')
        
        
print time.strftime('%a %H:%M:%S')


Sun 00:10:40
47000 : Sun 00:16:49
48000 : Sun 02:44:19
Sun 04:44:55


In [ ]:
#시작은 4시 9분이지만 실제 시작시간은 9월 28일 오후 5시 15분? 쯤임.
#시작은 목요일 7시 27분이지만 실제 시작시간은 목요일(29일) 8시 27분임.:ㅂ!
#시작은 일요일 12시 10분이지만 실제 시간은 일요일(2일) 오후 1시110
임.

In [118]:
df1 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_predicted_ontology.tsv")
df2 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_2_predicted_ontology_16062line.tsv")
df3 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_3_predicted_ontology_19721line.tsv")
df4 = pd.read_table("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_4_predicted_ontology_.tsv")

In [120]:
print len(df1)
print len(df2)
print len(df3)
print len(df4)

16060
2581
26098
1818


In [123]:
frames = [df1, df2, df3, df4]
df_all = pd.concat(frames)

In [125]:
df_all.head()

,substrate,product,predicted_ontology,predicted_score
0,65801,C05665,K08681,0.032
1,65801,C03375,K15575,0.056
2,65801,C03283,K06215,0.041
3,65801,C15853,K11780,0.072
4,65801,C01029,K08681,0.038


In [126]:
len(df_all)

46557

In [127]:
df_all.to_csv("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_all_predicted_ontolgy.tsv", sep='\t', index=False)

In [129]:
df_all.drop_duplicates().to_csv("/home/share/aschoi/data/COCONUT_Herbal/7_Ezyme_tool/3_all_predicted_ortolgy_wo_duplicate.tsv", sep='\t', index=False)

In [131]:
a = df_all.drop_duplicates()

In [136]:
df_all.index = range(0, len(df_all))

In [143]:
type(df_all['substrate'][0])

numpy.int64

In [142]:
df_all.substrate = df_all.compID.astype(int)

numpy.float64

In [ ]:

# 1. requests HTML

url_base = 'http://rest.genome.jp/ezyme/'

# list of r object(request result)
rs=[]
cnt = 0
print time.strftime('%a %H:%M:%S')
for index, row in herb_product_pair[len(rs):].iterrows():
    #try:
    sub = row['compID']
    pro = row['product']
    url = url_base + 'CID'+str(int(sub)) +'/' + pro
        #scrap the HTML at the url
    #r = requests.get(url)
        #rows = r.text.split('\n')
        #row = rows[1].split('\t')
        #predicted_score = row[0].encode('ascii', 'replace')
        #predicted_orthology = row[1].encode('ascii', 'replace')
        #rs.append(i+(predicted_orthology, predicted_score))
    s = requests.Session()
    r = s.get(url)
    rs.append(r)
    #r.connection.config['kepp_alive'] = False
    #r.conection.close()
    s.close()
    #s.config
    #s.config['keep_alive']=False
    #except requests.ConnectionError:
        #rs.append('null')
    #except urllib2.socket.error:
    
    
    #    print i
    #    print len(rs)
    #    print url
    #    print r
    #    print rs[len(rs)-1]
    #    break   
    cnt = cnt + 1
    if cnt % 1000 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Mon 06:27:35


In [143]:
row

compID      50516
product    C00458
Name: 40, dtype: object

In [139]:
len(rs)

2219

In [140]:
cnt

2219

In [112]:
s.get('http://rest.genome.jp/ezyme/').text

u''

In [128]:
r.connection.config['kepp_alive'] = False

In [130]:
r.connection.close()

In [132]:
s.close()

AttributeError: 'Session' object has no attribute 'text'

In [121]:
s = requests.session()

In [122]:
r = s.get(url)

In [124]:
s.close

<bound method Session.close of <requests.sessions.Session object at 0x7f328890ae90>>

In [118]:
url

'http://rest.genome.jp/ezyme/65801/C05665'

In [89]:
str(sub)

'65801.0'

In [91]:
url

NameError: name 'url' is not defined

In [94]:
pro

'C05665'